In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Group 9
# Amr Osama Abdelsalam
# Gouda Ebrahem Gouda
# Ahmed Elsaid Abdo
# Mohamed Amin Atallah

In [ ]:
import seaborn as sns

In [ ]:
train_df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')

In [ ]:
train_df.info()

### atemp is highly correlated to temp and casula and registered are missing in the test data

In [ ]:
train_df.drop(['atemp','casual','registered'],axis=1,inplace=True)

### extracting month and hour and week day features

In [ ]:
def extract_features(df):
    df.datetime = pd.to_datetime(df.datetime)
    df['hour'] = df.datetime.dt.hour
    df['month'] = df.datetime.dt.month
    df['weekday'] = df.datetime.dt.weekday


In [ ]:
extract_features(train_df)

In [ ]:
train_df.head()

In [ ]:
# sns.pairplot(data=train_df[['weekday','hour','month','count']],hue='count')

In [ ]:
train_df['count'].hist()

### fixing count skewing

In [ ]:
train_df['count'] = np.log1p(train_df['count'])

In [ ]:
train_df['count'].hist()

In [ ]:
# sns.pairplot(data=train_df,hue='count')

In [ ]:
train_df.columns.drop('count')

In [ ]:
train_df.describe().T

In [ ]:
corr = train_df.corr()

In [ ]:
sns.heatmap(corr,cmap='coolwarm',annot=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import ExtraTreesRegressor


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['count','datetime'],axis=1), train_df['count'], test_size=0.33, random_state=42)

In [ ]:

# poly = PolynomialFeatures(degree=3)
# X_train_ = poly.fit_transform(X_train)
# reg = LinearRegression().fit(X_train_,y_train)
reg = ExtraTreesRegressor(n_estimators=15, random_state=42).fit(X_train, y_train)

In [ ]:
reg.score(X_train,y_train)


In [ ]:
# X_test_ = poly.fit_transform(X_test)
reg.score(X_test,y_test)

In [ ]:
test_df = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
extract_features(test_df)

In [ ]:
x_test = test_df.drop(['datetime','atemp'],axis=1)

In [ ]:
x_test.head()

In [ ]:
# X_test__ = poly.fit_transform(x_test)
y_log = reg.predict(x_test)
y_result = np.expm1(y_log)

In [ ]:
y_result = np.expm1(y_log)

In [ ]:
result = pd.DataFrame({'datetime':test_df.datetime,'count':y_result})

In [ ]:
result['count']=result['count'].astype(int)


In [ ]:
result.to_csv('/kaggle/working/bike_share_prediction.csv',index=False)

In [ ]:
result